In [1]:
import pandas as pd
import re 
import nltk as nltk
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import spacy
import time
import math
import string
import gensim
import pyLDAvis
import seaborn as sns 
import matplotlib.pyplot as plt
#import pyLDAvis.gensim
#import pyLDAvis.gensim_models as gensimvis
import warnings
warnings.simplefilter('ignore')

from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from gensim import corpora, models
from gensim.models.ldamulticore import LdaMulticore

pyLDAvis.enable_notebook()
#nlp = spacy.load("en_core_web_sm")

In [2]:
import pandarallel
import multiprocessing
from pandarallel import pandarallel

num_processors = multiprocessing.cpu_count()

pandarallel.initialize(nb_workers=num_processors, use_memory_fs=False, progress_bar=True)

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [3]:
%%time

df = pd.read_parquet('filter_data.parquet', engine='pyarrow')
df.shape

CPU times: total: 125 ms
Wall time: 222 ms


(1650, 13)

### Topic Modelling 

In [4]:
df.head(5)

,date,title,text,year,month,day,dayofweek,clean_title,clean_text,clean_title_tokens,article_length,title_length,clean_text_tokens
0,2023-05-25,PayPal Is Using AI to Supercharge Profits | Na...,\nPayPal Is Using AI to Supercharge Profits | ...,2023,5,25,3,PayPal Is Using AI to Supercharge Profits Nasdaq,PayPal Is Using AI to Supercharge Profits Nasd...,"[paypal, using, ai, supercharge, profits, nasdaq]",7090,48,"[paypal, using, ai, supercharge, profits, nasd..."
1,2023-02-24,Facebook Parent Meta Wants to Show It's Still ...,\n\n\nFacebook Parent Meta Wants to Show It's ...,2023,2,24,4,Facebook Parent Meta Wants to Show Its Still a...,Facebook Parent Meta Wants to Show Its Still a...,"[facebook, parent, meta, wants, show, still, b...",4689,76,"[facebook, parent, meta, wants, show, still, b..."
2,2021-03-23,Artificial Intelligence Market is Expected to ...,\nArtificial Intelligence Market is Expected t...,2021,3,23,1,Artificial Intelligence Market is Expected to ...,Artificial Intelligence Market is Expected to ...,"[artificial, intelligence, market, expected, g...",6953,122,"[artificial, intelligence, market, expected, g..."
3,2022-07-23,Google fires employee who said its conversatio...,Google fires employee who said its conversatio...,2022,7,23,5,Google fires employee who said its conversatio...,Google fires employee who said its conversatio...,"[google, fire, employee, said, conversation, a...",5754,62,"[google, fire, employee, said, conversation, a..."
4,2021-06-22,Radically oversimplified AI skin-tone tool cou...,\n \n\nRadically oversimplified AI skin-tone t...,2021,6,22,1,Radically oversimplified AI skintone tool coul...,Radically oversimplified AI skintone tool cou...,"[radically, oversimplified, ai, skintone, tool...",6406,81,"[radically, oversimplified, ai, skintone, tool..."


In [5]:
df = df.drop(["text", "title"], axis=1)

In [6]:
# Creating a term dictionary and document term matrix
dictionary_n_title = corpora.Dictionary(df["clean_title_tokens"])
dictionary_n_text = corpora.Dictionary(df["clean_text_tokens"])

In [7]:
doc_term_matrix_n_title = [dictionary_n_title.doc2bow(doc) for doc in df["clean_title_tokens"]]
doc_term_matrix_n_text = [dictionary_n_text.doc2bow(doc) for doc in df["clean_text_tokens"]]

In [8]:
# Creating a Latent Dirichlet Allocation(LDA) Model for topic modelling
Lda = gensim.models.ldamodel.LdaModel

In [9]:
import multiprocessing
num_processors = multiprocessing.cpu_count()
num_processors

workers = num_processors

print(f'Using {workers} workers')

Using 12 workers


In [13]:
def initial_5_topic_mult(doc_term_matrix,dictionary):
    lda_model = LdaMulticore(corpus=doc_term_matrix,
                   id2word=dictionary,
                   num_topics=5,
                   random_state=100,
                   passes=10,
                   eta='auto',
                   workers=workers)

    for i in lda_model.print_topics(num_words=7):
        print(i)
    doc_lda = lda_model[doc_term_matrix]

In [14]:
%time initial_5_topic_mult(doc_term_matrix_n_title, dictionary_n_title)

(0, '0.034*"ai" + 0.011*"machine" + 0.010*"learning" + 0.006*"generative" + 0.006*"chatgpt" + 0.005*"launch" + 0.005*"news"')
(1, '0.045*"ai" + 0.013*"intelligence" + 0.013*"artificial" + 0.011*"new" + 0.007*"data" + 0.007*"science" + 0.006*"news"')
(2, '0.069*"ai" + 0.009*"chatgpt" + 0.006*"data" + 0.005*"launches" + 0.005*"new" + 0.005*"product" + 0.004*"hunt"')
(3, '0.068*"ai" + 0.013*"intelligence" + 0.013*"artificial" + 0.011*"news" + 0.007*"chatgpt" + 0.007*"new" + 0.007*"market"')
(4, '0.040*"ai" + 0.023*"intelligence" + 0.021*"artificial" + 0.016*"market" + 0.008*"technology" + 0.008*"news" + 0.007*"global"')
CPU times: total: 5.97 s
Wall time: 20.6 s


In [ ]:
%time initial_5_topic_mult(doc_term_matrix_n_text, dictionary_n_text)

In [ ]:
def initial_10_topic_mult(doc_term_matrix,dictionary):
    lda_model = LdaMulticore(corpus=doc_term_matrix,
                   id2word=dictionary,
                   num_topics=10,
                   random_state=100,
                   passes=10,
                   eta='auto',
                   workers=workers)

    for i in lda_model.print_topics(num_words=7):
        print(i)
    doc_lda = lda_model[doc_term_matrix]

In [19]:
%time initial_10_topic_mult(doc_term_matrix_n_text, dictionary_n_text)

(0, '0.010*"ai" + 0.008*"machine" + 0.008*"learning" + 0.008*"republic" + 0.006*"new" + 0.006*"market" + 0.005*"news"')
(1, '0.010*"ai" + 0.007*"news" + 0.007*"market" + 0.006*"industry" + 0.006*"public" + 0.005*"business" + 0.005*"technology"')
(2, '0.013*"data" + 0.012*"ai" + 0.007*"business" + 0.006*"customer" + 0.006*"news" + 0.005*"new" + 0.005*"company"')
(3, '0.011*"ai" + 0.007*"company" + 0.006*"stock" + 0.005*"ha" + 0.004*"new" + 0.004*"market" + 0.004*"news"')
(4, '0.012*"ago" + 0.008*"hour" + 0.007*"ai" + 0.006*"new" + 0.005*"news" + 0.005*"said" + 0.005*"day"')
(5, '0.008*"ai" + 0.006*"new" + 0.005*"intelligence" + 0.005*"ha" + 0.005*"artificial" + 0.004*"news" + 0.004*"data"')
(6, '0.012*"ai" + 0.007*"news" + 0.006*"data" + 0.005*"paid" + 0.005*"technology" + 0.004*"business" + 0.004*"program"')
(7, '0.013*"ai" + 0.007*"gray" + 0.006*"group" + 0.006*"technology" + 0.006*"media" + 0.005*"health" + 0.005*"patient"')
(8, '0.037*"market" + 0.017*"ai" + 0.012*"intelligence" + 0